### Linear Regression with Pytorch Built-in Functions

So, carrying on from the previous notebook, we are going to implement linear regression by utilizing the built-in functions of Pytorch.

The dataset we are going to implement on, is heights and weights dataset obtained from the following link: https://www.kaggle.com/tmcketterick/heights-and-weights

Now, I have a habit of importing all the necessary packages in the first cell of code, so please don't be frightened of them.

In [68]:
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.nn.functional import mse_loss
from torch.optim import SGD

Now first step is to load the data. I have the data in a csv file in disc. So, i will be utilizing the pandas package to import it.

In [69]:
df = pd.read_csv('height_weight.csv')

df.head() #displaying the first 5 rows of the dataset

,Height,Weight
0,1.47,52.21
1,1.50,53.12
2,1.52,54.48
3,1.55,55.84
4,1.57,57.20


Okay, nice. The data has been loaded. But, we need to separate the feature and the target, since they are together. Also, for implementations with pytorch built in functions we need to convert the data into tensors, which is possile through convertion to numpy array.

So, lets do that.

In [71]:
height = df['Height'].to_numpy(dtype = 'float32')
weight = df['Weight'].to_numpy(dtype = 'float32')

We have converted the data into numpy array. Lets check it.

In [72]:
height

array([1.47, 1.5 , 1.52, 1.55, 1.57, 1.6 , 1.63, 1.65, 1.68, 1.7 , 1.73,
       1.75, 1.78, 1.8 , 1.83], dtype=float32)

In [73]:
weight

array([52.21, 53.12, 54.48, 55.84, 57.2 , 58.57, 59.93, 61.29, 63.11,
       64.47, 66.28, 68.1 , 69.92, 72.19, 74.46], dtype=float32)

One more thing we have to do is to the expand the dimensions of the height and weight.

In [74]:
height = np.expand_dims(height , axis = 1)
weight = np.expand_dims(weight , axis = 1)

Yes. We have done it correctly.

Next up, lets convert the numpy array to torch tensors using the method <code>torch.from_numpy()</code>

In [75]:
inputs = torch.from_numpy(height)  ##convertion of height to tensors

targets = torch.from_numpy(weight)  ##convertion of weight to tensors

Lets check if the convertion is okay!

In [76]:
inputs

tensor([[1.4700],
        [1.5000],
        [1.5200],
        [1.5500],
        [1.5700],
        [1.6000],
        [1.6300],
        [1.6500],
        [1.6800],
        [1.7000],
        [1.7300],
        [1.7500],
        [1.7800],
        [1.8000],
        [1.8300]])

In [77]:
targets

tensor([[52.2100],
        [53.1200],
        [54.4800],
        [55.8400],
        [57.2000],
        [58.5700],
        [59.9300],
        [61.2900],
        [63.1100],
        [64.4700],
        [66.2800],
        [68.1000],
        [69.9200],
        [72.1900],
        [74.4600]])

Yes! We have done it correctly.

Now moving on!

In general for the creation of a linear model we set up the weights and biases, as you have seen in my previous notebook.

But, we can skip that part and let Pytorch do that, by utilizing its <code>nn.Linear</code> module.

So, lets do that only.

In [78]:
linear_model = nn.Linear(1 , 1)

print(linear_model.weight)
print(linear_model.bias)
print(linear_model.parameters)

Parameter containing:
tensor([[-0.7696]], requires_grad=True)
Parameter containing:
tensor([-0.1375], requires_grad=True)
<bound method Module.parameters of Linear(in_features=1, out_features=1, bias=True)>


Boom! The model is set.

Now what do we do?

We set the loss function. Now, in our previous notebook, we created a manual loss function, but in here we are going to partake in a Pytorch's inbuilt mean squared error loss function through <code>mse_loss</code> (imported earlier from <code>torch.nn.functional</code>

In [81]:
loss_func = mse_loss

In [93]:
loss_func(linear_model(inputs[0]) , targets[0]).item()

2838.9990234375

Done! What next? Well we need to update our weights right? 
How do we do that? Well using an optimizer. Here we are going to utilize Stochastic Gradient Descent optimizer from Pytorch package.

In [82]:
optim = SGD(linear_model.parameters() , lr = 1e-5)

Now lets pull it all together, but, before that we are going to set up our training based on Pytorch Datasets and load it via Pytorch Dataloader. So, lets do that.

In [83]:
dataset = TensorDataset(inputs , targets)

dataset[0:3]  #for checking

(tensor([[1.4700],
         [1.5000],
         [1.5200]]),
 tensor([[52.2100],
         [53.1200],
         [54.4800]]))

Yep! The dataset is instantiated.

Now lets set the datalaoder.

In [84]:
dataloader = DataLoader(dataset , batch_size = 5 , shuffle = True)

In [85]:
for x, y in dataloader:
    print(x)

tensor([[1.7300],
        [1.4700],
        [1.8300],
        [1.7000],
        [1.5700]])
tensor([[1.6300],
        [1.5500],
        [1.7500],
        [1.6800],
        [1.7800]])
tensor([[1.5000],
        [1.6000],
        [1.6500],
        [1.5200],
        [1.8000]])


Done!

Now, lets move on and train our model through a custom function.

In [103]:
def fit(num_epochs , linear_model , dataloader , loss_func , optim):
    
    for epoch in range(num_epochs):
        
        for x, y in dataloader:
            
            ##making predictions
            y_pred = linear_model(x)
            
            ##checking the loss
            loss = loss_func(y_pred , y)
            
            ##backpropagating
            loss.backward()
            
            ##optimizing the parameters
            optim.step()
            
            ##resetting the optimizer
            optim.zero_grad()
            
        if (epoch + 1) % 5 == 0:
            print('Epoch [{} / {}] , Loss : {:.2f}'.format(epoch + 1 , num_epochs , loss.item()))

Lets train our model by calling the fit function.

In [104]:
fit(100 , linear_model , dataloader , loss_func , optim)

Epoch [5 / 100] , Loss : 3351.71
Epoch [10 / 100] , Loss : 4158.58
Epoch [15 / 100] , Loss : 4302.25
Epoch [20 / 100] , Loss : 4037.56
Epoch [25 / 100] , Loss : 3684.95
Epoch [30 / 100] , Loss : 4015.62
Epoch [35 / 100] , Loss : 3542.15
Epoch [40 / 100] , Loss : 3366.64
Epoch [45 / 100] , Loss : 3419.83
Epoch [50 / 100] , Loss : 4264.02
Epoch [55 / 100] , Loss : 3259.37
Epoch [60 / 100] , Loss : 3693.41
Epoch [65 / 100] , Loss : 4178.17
Epoch [70 / 100] , Loss : 3731.56
Epoch [75 / 100] , Loss : 4116.34
Epoch [80 / 100] , Loss : 3821.48
Epoch [85 / 100] , Loss : 4049.86
Epoch [90 / 100] , Loss : 3095.14
Epoch [95 / 100] , Loss : 3889.48
Epoch [100 / 100] , Loss : 3594.03


And Boom Done!!